In [5]:
from util import *
from util import Psr_query

blimpy.waterfall WARNING  Could not import bitshuffle, using hdf5plugin failover. Write access disabled.


In [ ]:
from blimpy import Waterfall
import pandas as pd
import h5py
PSR_DATA = pd.read_csv("object_GBT_pulsars.csv")
PSR_LST = list(PSR_DATA["NAME"])
PSR_LST = PSR_LST # + list(map(lambda x: "DIAG_"+x,PSR_LST))
QUERY_API = "http://seti.berkeley.edu/opendata//api/query-files"
DATA = pd.read_csv("psrdata.csv")
Rpath = "/mnt_blpd13/datax/cal_and_psrs/"
servername ="shufanl@blpl1.ssl.berkeley.edu"


import requests
class Psr_query:
    psr_observations = None
    @staticmethod
    def write_score(filename, score):
        f = h5py.File(filename)
        f['data'].attrs['quality'] = score
        f.close()

    @staticmethod
    def read_score(filename):
        f = h5py.File(filename)
        result = f['data'].attrs.get('quality')
        f.close()
        return result

    def __init__(self):
        self.data = DATA
        pass
    @staticmethod
    def get_observation_time(file):
        f = Waterfall(file)
        header = f.header
        return header[b'tstart']

    @property
    def get_psr_observations(self):
        if self.psr_observations:
            return self.psr_observations
        payload = {"file-types":"filterbank"}

        rst = []
        visited = set()
        for i in PSR_LST:
            payload["target"] = i
            r = requests.get(QUERY_API,payload)
            js = r.json()
            print(len(js.get("data",[])).__str__()+" obserivations for "+ i)
            for entry in js.get("data",[]):
                md5 = entry["md5sum"]
                if md5 not in visited:
                    visited.add(md5)
                    rst.append(entry)
                    #print(entry)
            #break

        rst.sort(key = lambda x:x['size'],reverse = True)
        rst.sort(key = lambda x:x['mjd'],reverse = True)
        self.psr_observations = rst
        return rst

    def latest_psr(self,file):
        t_start = self.get_observation_time(file)
        return Rpath+self.get_closest_observations(self.data,t_start)

    def calculate_SNR(self):
        pass

    def get_closest_observations(self,df, mjd):
        df["time_delta"] = 0
        for index in df.index:
            print(df.loc[index, "tstart"],mjd )
            df.loc[index, "time_delta"] = abs(df.loc[index, "tstart"] - mjd)
        m = df.sort_values("time_delta")
        k = m.loc[1]
        print("DIFF:"+str(k["time_delta"]))
        return k["fname"]


In [7]:
testfilename = "GBT_57438_42346_J1239+2453_mid.h5"
testfilepath = "/mnt_blpd13/datax/cal_and_psrs/"
servername ="shufanl@blpl1.ssl.berkeley.edu"

In [140]:
readdatascore(testfilename)

100

In [141]:
writedatascore(testfilename,100)

In [142]:
h = Psr_query()

In [ ]:
servername+":"+h.latest_psr(testfilename)

In [144]:
a = pd.read_csv("psrdata.csv")

In [145]:
a

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,az_start,data_type,fch1,foff,ibeam,machine_id,nbeams,...,nifs,rawdatafile,source_name,src_dej,src_raj,telescope_id,tsamp,tstart,za_start,fname
0,556,556,556,0.0,1,8438.964844,-0.000003,NaN,20,NaN,...,1,NaN,PSR_B1929+10,10d59m32.64s,19h32m13.848s,6,18.253611,58921.362014,0.0,spliced_blc10111213141516o7o0212223242526o7o03...
1,564,564,564,0.0,1,8438.964844,-0.366211,0.0,10,0.0,...,1,NaN,PSR_B1929+10,10d59m32.64s,19h32m13.848s,6,0.000350,58921.362014,0.0,spliced_blc10111213141516o7o0212223242526o7o03...
2,563,563,563,0.0,1,8438.964844,-0.002861,NaN,20,NaN,...,1,NaN,PSR_B1929+10,10d59m32.64s,19h32m13.848s,6,1.073742,58921.362014,0.0,spliced_blc10111213141516o7o0212223242526o7o03...
3,562,562,562,0.0,1,8438.964844,-0.366211,NaN,20,NaN,...,1,NaN,PSR_B1929+10,10d59m32.64s,19h32m13.848s,6,0.000350,58921.362014,0.0,spliced_blc10111213141516o7o0212223242526o7o03...
4,262,262,262,0.0,1,8438.964844,-0.366211,0.0,10,0.0,...,1,NaN,PSR_B2021+51,51d54m50.4s,20h22m49.8s,6,0.000350,58921.340625,0.0,spliced_blc10111213141516o7o0212223242526o7o03...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,187,187,187,0.0,1,3151.281738,-0.366211,NaN,20,NaN,...,1,NaN,PSR_B1133+16,19d20m26.16s,12h49m38.448s,6,0.000350,58538.308993,0.0,spliced_blc4041424344454647_guppi_58538_26697_...
160,305,305,305,0.0,1,11251.463413,-0.002861,NaN,20,NaN,...,1,NaN,PSR_B0355+54,54d12m46.8s,3h58m55.368s,6,1.073742,58532.673067,0.0,spliced_blc40414243444546o7o0515253545556o7o06...
161,303,303,303,0.0,1,11251.281738,-0.366211,NaN,20,NaN,...,1,NaN,PSR_B0355+54,54d12m46.8s,3h58m55.368s,6,0.000350,58532.673067,0.0,spliced_blc40414243444546o7o0515253545556o7o06...
162,314,314,314,0.0,1,11251.281738,-0.366211,0.0,10,0.0,...,1,NaN,PSR_B0355+54,54d12m46.8s,3h58m55.368s,6,0.000350,58532.673067,0.0,spliced_blc40414243444546o7o0515253545556o7o06...


In [146]:
c = h.get_psr_observations

369 obserivations for PSR_J0332+5434
24 obserivations for PSR_J0953+0755
0 obserivations for PSR_J1935+1616
0 obserivations for PSR_J2022+2854
0 obserivations for PSR_J1932+1059
0 obserivations for PSR_J2018+2839
0 obserivations for PSR_J2022+5154
0 obserivations for PSR_J0358+5413
0 obserivations for PSR_J1645-0317
0 obserivations for PSR_J1807-0847
100 obserivations for PSR_J0534+2200
0 obserivations for PSR_J0248+6021
0 obserivations for PSR_J1939+2134
0 obserivations for PSR_J0454+5543
0 obserivations for PSR_J2048-1616
0 obserivations for PSR_J1713+0747
0 obserivations for PSR_J0814+7429
0 obserivations for PSR_J0826+2637
0 obserivations for PSR_J1239+2453
0 obserivations for PSR_J1740+1000
0 obserivations for PSR_J2257+5909
0 obserivations for PSR_J0543+2329
0 obserivations for PSR_J2145-0750
0 obserivations for PSR_J1844+0000
0 obserivations for PSR_J1848-0123
0 obserivations for PSR_J1948+3540
0 obserivations for PSR_J1509+5531
0 obserivations for PSR_J1705-1906
0 obserivations

In [130]:
c = pd.DataFrame(c)

In [137]:
c

,center_freq,decl,file_type,md5sum,mjd,ra,size,target,telescope,url,utc,tstart
0,1501.464842,54.57900,filterbank,cd6d683a5df5638cf36a4643aba973fa,58110.073252,53.24750,34359738675,DIAG_PSR_J0332+5434,GBT,http://blpd0.ssl.berkeley.edu/sample_data/spli...,"Sat, 23 Dec 2017 01:45:29 GMT",58110.073252
1,1501.281738,54.57900,filterbank,1d4385bd3f79b7da1b034909dbd27f2f,58110.073252,53.24750,14059307315,DIAG_PSR_J0332+5434,GBT,http://blpd0.ssl.berkeley.edu/sample_data/spli...,"Sat, 23 Dec 2017 01:45:29 GMT",58110.073252
2,2157.714842,54.57900,filterbank,3a3378280fdad8bb916a76175af00c7b,58110.073252,53.24750,4294967700,DIAG_PSR_J0332+5434,GBT,http://blpd0.ssl.berkeley.edu/sample_data/raw_...,"Sat, 23 Dec 2017 01:45:29 GMT",58110.073252
3,1970.214842,54.57900,filterbank,93fdf514df9bb973518eaa855c0f628a,58110.073252,53.24750,4294967700,DIAG_PSR_J0332+5434,GBT,http://blpd0.ssl.berkeley.edu/sample_data/raw_...,"Sat, 23 Dec 2017 01:45:29 GMT",58110.073252
4,1782.714842,54.57900,filterbank,aff4d176837c4a8c11973e669b379486,58110.073252,53.24750,4294967700,DIAG_PSR_J0332+5434,GBT,http://blpd0.ssl.berkeley.edu/sample_data/raw_...,"Sat, 23 Dec 2017 01:45:29 GMT",58110.073252
...,...,...,...,...,...,...,...,...,...,...,...,...
431,2401.463413,38.76200,filterbank,ecb7d096effcc59e06eb57d528ce9c5a,57774.850741,303.29292,570425651,DIAG_PSR_J2013+3845,GBT,http://blpd0.ssl.berkeley.edu/LHS1140/PSR/spli...,"Sat, 21 Jan 2017 20:25:04 GMT",57774.850741
432,1501.464842,38.76211,filterbank,722d774e420b680282b75cf56f8d2815,57774.810463,303.29292,34359738675,DIAG_PSR_J2013+3845,GBT,http://blpd0.ssl.berkeley.edu/LHS1140/PSR/spli...,"Sat, 21 Jan 2017 19:27:04 GMT",57774.810463
433,1501.281738,38.76211,filterbank,68366bf474c880e826533796310c02bd,57774.810463,303.29292,13723762995,DIAG_PSR_J2013+3845,GBT,http://blpd0.ssl.berkeley.edu/LHS1140/PSR/spli...,"Sat, 21 Jan 2017 19:27:04 GMT",57774.810463
434,1501.281738,38.76211,filterbank,577910d09200d91173f01d2756ced3ed,57774.810463,303.29292,3430941025,DIAG_PSR_J2013+3845,GBT,http://blpd0.ssl.berkeley.edu/LHS1140/PSR/spli...,"Sat, 21 Jan 2017 19:27:04 GMT",57774.810463


In [134]:
c.to_csv("psrdump.csv")

In [63]:
def get_closest_observations(df,mjd):
    df["time_delta"] =0
    for index in df.index:
        df.loc[index,"time_delta"]=abs(df.loc[index,"tstart"]-mjd)
    return df.sort_values("time_delta").loc[0]["fname"]

In [ ]:
get_closest_observations(a,57921)

In [86]:
a["nbits"]

0      32
1       8
2      32
3      32
4       8
       ..
159    32
160    32
161    32
162     8
163    32
Name: nbits, Length: 164, dtype: int64

In [12]:
a = Waterfall("spliced_blc7071727374757677_guppi_58845_40114_PSR_B1133+16_0009.gpuspec.0002.fil")
t = "spliced_blc7071727374757677_guppi_58845_40114_PSR_B1133+16_0009.gpuspec.0002.fil"

In [ ]:
a.plot_waterfall().show()

In [ ]:
matplotlib.use('TkAgg')

In [ ]:
import matplotlib

In [17]:
matplotlib.use('TkAgg')

In [14]:
#!/bin/python
import os, sys
import pandas as pd

# ----- Config ----- #
Np = 2
Ttime = 5 * 60  # In seconds
Wfrac = 5  # Assuming 5% pulse width


# csvfile = "/home/obs/target_logs/PULSARS"
#
# ------------------ #

def process(filfile):
    fname = filfile.split('/')[-1].strip('.fil')

    # Get Session ID
    SessionID = '*'
    tmp = "_".join(fname.split("_")[1:4])
    if tmp[:4] == "AGBT": SessionID = tmp

    # print SessionID

    # Read CSV file

    cmd = "echo %s | sed 's/Psr/PSR/' | sed 's/psr/PSR/' | awk -F_PSR_ '{print $2}' | awk -F_ '{print $1}' | sed 's/j/J/' | sed 's/b/B/'" % (
        filfile)
    PSRNAME = os.popen(cmd).readline().strip()
    print(PSRNAME)
    # print PSRNAME

    cmd = "psrcat -e %s > tmp.par" % (PSRNAME)
    os.system(cmd)

    cmd = "dspsr -E tmp.par %s -O %s" % (filfile, fname)
    print(cmd)
    c = os.system(cmd)

    cmd = "paz -r -L -b -d -m %s.ar" % (fname)
    os.system(cmd)

    cmd = "psrstat -jDF -c 'snr=modular:{on=minimum:{find_min=0,smooth:width=0.05}}' -c snr %s.ar " % (fname)
    print(cmd)
    SNR = float(os.popen(cmd).readline().strip().split("=")[1])

    # Off pulse rms
    cmd = "psrstat -jDF -c 'snr=modular:{on=minimum:{find_min=0,smooth:width=0.05}}' -c off:rms %s.ar" % (fname)
    RMS = float(os.popen(cmd).readline().strip().split("=")[1])

    # Center frequency
    # FREQ = pfdfl.lofreq - 0.5*pfdfl.chan_wid + pfdfl.chan_wid*(pfdfl.numchan/2)
    cmd = "psredit -c freq %s.ar" % (fname)
    FREQ = float(os.popen(cmd).readline().strip().split("=")[1])

    cmd = "psredit -c bw %s.ar" % (fname)
    BW = float(os.popen(cmd).readline().strip().split("=")[1])
    BWMHz = BW
    BW = abs(BW * pow(10, 6))  # In Hz

    # Get MJD
    # cmd = "header %s -tstart" % (filfile)
    # MJD = os.popen(cmd).readline().strip()
    # print(SNR)

    BWfact = 0.9

    # SEFD
    if FREQ < 2000:
        SEFD = 10  # For GBT-L band
        BWfact = 0.6  # For GBT-BL only uses about 60% of the band
    if FREQ > 2000 and FREQ < 3000: SEFD = 12  # GBT-S band
    if FREQ > 3000 and FREQ < 8000: SEFD = 10  # GBT-C band
    if FREQ > 8000 and FREQ < 12000: SEFD = 15  # GBT-X band
    if FREQ > 12000 and FREQ < 18000: SEFD = 15  # GBT Ku-band
    if FREQ > 18000 and FREQ < 26000: SEFD = 25  # GBT KFPA-band
    if FREQ > 26000: raise ValueError('Need SEFD value for FREQ>26 GHz.')

    # Only for 80% of the band, we have sufficient sensitivity.
    BW = BWfact * BW

    # Get spectral index
    cmd = "psrcat -c 'SPINDX' -o short -nohead -nonumber " + str(PSRNAME) + " 2>&1 "
    SPINDEX = os.popen(cmd).readline().strip()

    # If not in the catalogue, then assume -1.4 (Bates et al. 2013)
    if SPINDEX == "*":
        SPINDEX = -1.4
    else:
        SPINDEX = float(SPINDEX)

    # Flux at 1400 MHz
    cmd = "psrcat -c 'S1400' -o short -nohead -nonumber " + str(PSRNAME) + " 2>&1 "
    S1400 = float(os.popen(cmd).readline().strip())

    # Flux at the observing frequency
    FLUX = S1400 * pow((FREQ / 1400.0), SPINDEX)

    # Expected average profile SNR
    expSNR = FLUX * pow(10, -3) * pow(Np * Ttime * BW, 0.5) / (1.16 * SEFD)
    print(SNR)
    print(expSNR)
    return {"SNR": SNR, "expSNR": expSNR}


In [15]:
k = process(t)

B1133+16
dspsr -E tmp.par spliced_blc7071727374757677_guppi_58845_40114_PSR_B1133+16_0009.gpuspec.0002.fil -O spliced_blc7071727374757677_guppi_58845_40114_PSR_B1133+16_0009.gpuspec.0002
psrstat -jDF -c 'snr=modular:{on=minimum:{find_min=0,smooth:width=0.05}}' -c snr spliced_blc7071727374757677_guppi_58845_40114_PSR_B1133+16_0009.gpuspec.0002.ar 
nan
575.586996076098


{'SNR': nan, 'expSNR': 575.586996076098}

In [ ]:
k = abs()